In [17]:
from pyvis.network import Network
import pandas as pd
import json

In [73]:
wlan_df = pd.read_json('../api_flats/devices/phy-IEEE802.11.json')
wlan_df = wlan_df[[
    'kismet.device.base.key', 
    'kismet.device.base.name', 
    'kismet.device.base.type', 
    'kismet.device.base.packets.total', 
    'kismet.device.base.manuf', 
    'kismet.device.base.macaddr', 
    'kismet.device.base.channel', 
    'kismet.device.base.first_time', 
    'kismet.device.base.last_time',
    'dot11.device'
    ]]

wlan_df.rename(columns={
    'kismet.device.base.key': 'key', 
    'kismet.device.base.name': 'device_name', 
    'kismet.device.base.type': 'device_type', 
    'kismet.device.base.packets.total': 'num_packets', 
    'kismet.device.base.manuf': 'manuf', 
    'kismet.device.base.macaddr': 'macaddr', 
    'kismet.device.base.channel': 'channel', 
    'kismet.device.base.first_time': 'first_seen', 
    'kismet.device.base.last_time': 'last_seen',
    'dot11.device': 'dot11_device'
    }, inplace=True)

wlan_df_ap = wlan_df[wlan_df['device_type'] == 'Wi-Fi AP']
wlan_df_client = wlan_df[wlan_df['device_type'] == 'Wi-Fi Client']
wlan_df_client['ap_macaddr'] = wlan_df_client.apply(lambda row: row['dot11_device']['dot11.device.last_bssid'], axis=1)

/tmp/ipykernel_999/3961119332.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wlan_df_client['ap_macaddr'] = wlan_df_client.apply(lambda row: row['dot11_device']['dot11.device.last_bssid'], axis=1)


In [76]:
def get_probed_ssid(x):
    try:
        probed_ssid_map = x['dot11.device.probed_ssid_map']
    except:
        return None
    
    probed_ssids = []
    for val in probed_ssid_map:
        probed_ssids.append(val['dot11.probedssid.ssid'])

    return probed_ssids

def get_advertised_ssid(x):
    try:
        advertised_ssid = x['dot11.device.advertised_ssid_map'][0]['dot11.advertisedssid.ssid']
    except:
        return None

    return advertised_ssid
    
wlan_df_client['probed_ssids'] = wlan_df_client.apply(lambda row: get_probed_ssid(row['dot11_device']), axis=1)
wlan_df_client['probed_ssids_len'] = wlan_df_client.apply(lambda row: len(row['probed_ssids']) if row['probed_ssids']!=None else 0, axis=1)
wlan_df_ap['advertised_ssid'] = wlan_df_ap.apply(lambda row: get_advertised_ssid(row['dot11_device']), axis=1)

/tmp/ipykernel_999/2501399549.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wlan_df_client['probed_ssids'] = wlan_df_client.apply(lambda row: get_probed_ssid(row['dot11_device']), axis=1)
/tmp/ipykernel_999/2501399549.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wlan_df_client['probed_ssids_len'] = wlan_df_client.apply(lambda row: len(row['probed_ssids']) if row['probed_ssids']!=None else 0, axis=1)
/tmp/ipykernel_999/2501399549.py:23: SettingWithCopyWarning: 
A value is trying to be set on a

In [78]:
wlan_df = pd.merge(wlan_df_client, wlan_df_ap[['macaddr', 'advertised_ssid']], left_on='ap_macaddr', right_on='macaddr', how='left')
del wlan_df['macaddr_y']
del wlan_df['dot11_device']
wlan_df.rename(columns={'macaddr_x': 'macaddr', 'advertised_ssid': 'connected_ssid'}, inplace=True)
wlan_df.sort_values(by=['probed_ssids_len'], ascending=False)

,key,device_name,device_type,num_packets,manuf,macaddr,channel,first_seen,last_seen,ap_macaddr,probed_ssids,probed_ssids_len,connected_ssid
3124,4202770D00000000_1957EF9AEF2E,,Wi-Fi Client,20,Unknown,2E:EF:9A:EF:57:19,161,2023-04-22 02:09:49,2023-04-22 02:09:49,00:00:00:00:00:00,"[Dunkin' Donuts Guest, , TP-Link_2849, Airwave...",10,NaN
6513,4202770D00000000_CBE9DE280846,,Wi-Fi Client,110,Unknown,46:08:28:DE:E9:CB,132,2023-04-22 09:21:59,2023-04-22 09:21:59,2C:5D:93:6F:D5:4C,"[, PANERA, The Standard, Airwave-5G-Clubhouse,...",10,Airwave-5G-4-cww4sv0vs2
2669,4202770D00000000_C48AB5332302,,Wi-Fi Client,19,Unknown,02:23:33:B5:8A:C4,44,2023-04-22 01:24:24,2023-04-22 01:24:24,00:00:00:00:00:00,"[Airwave-5G-2-bz2y1vx0, RIC, , BRAZOS-GUEST, T...",9,NaN
4879,4202770D00000000_DB6FEFE833B6,,Wi-Fi Client,11,Unknown,B6:33:E8:EF:6F:DB,36,2023-04-22 05:14:49,2023-04-22 05:14:49,00:00:00:00:00:00,"[, The Standard, Airwave-5G-Clubhouse, Starbuc...",9,NaN
4883,4202770D00000000_A07AB56ADDC6,,Wi-Fi Client,14,Unknown,C6:DD:6A:B5:7A:A0,153,2023-04-22 05:14:52,2023-04-22 05:14:52,00:00:00:00:00:00,"[, PANERA, Airwave-5G-Clubhouse, FlatsBusiness...",8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730,4202770D00000000_D7E2380A86E,,Wi-Fi Client,8,Unknown,6E:A8:80:23:7E:0D,,2023-04-22 01:32:09,2023-04-22 01:32:09,2C:5D:93:2F:CD:8C,None,0,Airwave-4-cw3vw1w0
2729,4202770D00000000_9EFDB0FC4FBA,,Wi-Fi Client,7,Unknown,BA:4F:FC:B0:FD:9E,149,2023-04-22 01:32:03,2023-04-22 01:32:35,00:25:00:FF:94:73,None,0,NaN
2728,4202770D00000000_73FF51F932F6,,Wi-Fi Client,2,Unknown,F6:32:F9:51:FF:73,149,2023-04-22 01:31:55,2023-04-22 01:32:03,00:25:00:FF:94:73,None,0,NaN
2727,4202770D00000000_FBB0AC31687E,,Wi-Fi Client,4,Unknown,7E:68:31:AC:B0:FB,149,2023-04-22 01:31:35,2023-04-22 01:31:43,00:25:00:FF:94:73,None,0,NaN


In [79]:
wlan_df['probed_ssids_len'].value_counts()

probed_ssids_len
0     6004
1     1130
2       28
7        7
3        5
4        3
5        3
6        2
9        2
10       2
8        1
Name: count, dtype: int64